In [1]:
%pip install -Uq transformers hybridtfidf spacy

Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import pipeline
from hybridtfidf import HybridTfidf
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import spacy
nlp = spacy.load("en_core_web_sm")
hybridtfidf = HybridTfidf(threshold=7)

/home/mickey-pech/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-25 09:32:46.808364: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-25 09:32:46.894904: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-25 09:32:46.894968: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-25 09:32:46.897454

In [3]:
pipe = pipeline(model="microsoft/DialoGPT-medium")

/home/mickey-pech/.local/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

In [5]:
PREFIX = "STOIC: "
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "howdy", "ghbdtn", "privet", "hey", "toki", "pona")
GOODBYE_INPUTS = ("goodbye", "bye", "quit", "exit", "poka", "mi tawa")

In [6]:
def initialize() -> None:
    chat_history_ids = []
    
    with open("./stoic_data.txt", "r") as stoic_file:
        sentences = stoic_file.readlines()
    replies = sentences

    sent_tokens = []
    for sent in sentences:
        lemm_sent = [word.lemma_ for word in nlp(sent) if not word.is_stop and word.text.isalpha()]
        sent_tokens.append(" ".join(lemm_sent))

    print("Initialized successfully!\n_________________________\n\n")
    return sent_tokens, replies, chat_history_ids

sent_tokens, replies, chat_history_ids = initialize()

Initialized successfully!
_________________________




In [9]:
def response(user_response: str, sent_tokens: list, replies: list) -> str:
    unedited_user_response = user_response

    user_response = " ".join([word.lemma_ for word in nlp(user_response) \
                            if not word.is_stop and word.text.isalpha()])
    sent_tokens.append(user_response)

    hybridtfidf.fit(sent_tokens)

    tfidf = np.array(hybridtfidf.transform(sent_tokens))
    
    vals = cosine_similarity(tfidf[-1].reshape(1, -1), tfidf)
    
    idx = vals.argsort()[0][-2]
    
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    
    if(req_tfidf < 0.5):
        robo_response = model_layer(unedited_user_response, chat_history_ids)
    else:
        robo_response = replies[idx]
        
    sent_tokens.remove(user_response)
    return robo_response

def model_layer(user_input, chat_history_ids):
    for step in range(5):
        # encode the new user input, add the eos_token and return a tensor in Pytorch
        new_user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

        # append the new user input tokens to the chat history
        bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

        # generated a response while limiting the total chat history to 1000 tokens, 
        chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

        # pretty print last ouput tokens from bot
        return tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

def main() -> None:
    user_response = input("YOU: ").lower()
    chat_history_ids.append(user_response)

    if user_response in GREETING_INPUTS:
        print(f"{PREFIX}Greetings!")
        main()

    if user_response in GOODBYE_INPUTS:
        print(f"{PREFIX}Goodbye!")
        quit()
    
    if "thanks" in user_response or "thank you" in user_response:
        print(f"{PREFIX}You are welcome!")
        main()
    
    print(response(user_response, sent_tokens, replies))
    main()


In [10]:
main()

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


A philosophy that is not stoicism.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


A philosophy that is not stoicism.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



STOIC: Goodbye!


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


KeyboardInterrupt: 

: 